In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
# input dataset
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=['date'])
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=['date'])
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv', parse_dates=['date'])

In [3]:
# adjust date formats
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
holidays['date'] = pd.to_datetime(holidays['date'])
oil['date'] = pd.to_datetime(oil['date'])

# merge the dataset
train = train.merge(stores, on='store_nbr', how='left')
test = test.merge(stores, on='store_nbr', how='left')

train = train.merge(transactions, on=['date', 'store_nbr'], how='left')
test = test.merge(transactions, on=['date', 'store_nbr'], how='left')

train = train.merge(oil, on='date', how='left')
test = test.merge(oil, on='date', how='left')

holidays = holidays[['date', 'locale', 'transferred']].rename(columns={'locale': 'holidays_locale'})
train = train.merge(holidays, on='date', how='left', suffixes=('', '_holiday'))
test = test.merge(holidays, on='date', how='left', suffixes=('', '_holiday'))

train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 14 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id               int64         
 1   date             datetime64[ns]
 2   store_nbr        int64         
 3   family           object        
 4   sales            float64       
 5   onpromotion      int64         
 6   city             object        
 7   state            object        
 8   type             object        
 9   cluster          int64         
 10  transactions     float64       
 11  dcoilwtico       float64       
 12  holidays_locale  object        
 13  transferred      object        
dtypes: datetime64[ns](1), float64(3), int64(4), object(6)
memory usage: 326.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----       

In [4]:
# check the missing values
print("---missing value---")
print(train.isnull().sum())
print(test.isnull().sum())

print("\n---unique value---")
for col in train:
    print(f"{col}:{train[col].nunique()}")

---missing value---
id                       0
date                     0
store_nbr                0
family                   0
sales                    0
onpromotion              0
city                     0
state                    0
type                     0
cluster                  0
transactions        249117
dcoilwtico          955152
holidays_locale    2551824
transferred        2551824
dtype: int64
id                     0
date                   0
store_nbr              0
family                 0
onpromotion            0
city                   0
state                  0
type                   0
cluster                0
transactions       28512
dcoilwtico          7128
holidays_locale    26730
transferred        26730
dtype: int64

---unique value---
id:3000888
date:1684
store_nbr:54
family:33
sales:379610
onpromotion:362
city:22
state:16
type:5
cluster:17
transactions:4993
dcoilwtico:994
holidays_locale:3
transferred:2


In [5]:
#fill in the missing value
train['holidays_locale'] = train['holidays_locale'].fillna(False)
test['holidays_locale'] = test['holidays_locale'].fillna(False)

train['transferred'] = train['transferred'].fillna(False)
test['transferred'] = test['transferred'].fillna(False)

train['transactions'] = train.groupby('store_nbr')['transactions'].transform(lambda x: x.fillna(x.median()))
store_medians = train.groupby('store_nbr')['transactions'].median()
test['transactions'] = test['store_nbr'].map(store_medians)

train['dcoilwtico'] = train['dcoilwtico'].interpolate(method='linear')
test['dcoilwtico'] = test['dcoilwtico'].interpolate(method='linear')
train['dcoilwtico'] = train['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')
test['dcoilwtico'] = test['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')

/tmp/ipykernel_13/2010465726.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['transferred'] = train['transferred'].fillna(False)
/tmp/ipykernel_13/2010465726.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['transferred'] = test['transferred'].fillna(False)
/tmp/ipykernel_13/2010465726.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['dcoilwtico'] = train['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_13/2010465

In [6]:
# check the missing values
print("---missing value---")
print(train.isnull().sum())
print(test.isnull().sum())

print("\n---unique value---")
for col in train:
    print(f"{col}:{train[col].nunique()}")

---missing value---
id                 0
date               0
store_nbr          0
family             0
sales              0
onpromotion        0
city               0
state              0
type               0
cluster            0
transactions       0
dcoilwtico         0
holidays_locale    0
transferred        0
dtype: int64
id                 0
date               0
store_nbr          0
family             0
onpromotion        0
city               0
state              0
type               0
cluster            0
transactions       0
dcoilwtico         0
holidays_locale    0
transferred        0
dtype: int64

---unique value---
id:3000888
date:1684
store_nbr:54
family:33
sales:379610
onpromotion:362
city:22
state:16
type:5
cluster:17
transactions:5001
dcoilwtico:933305
holidays_locale:4
transferred:2


In [7]:
#extract temporal feature
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['dayofweek'] = train['date'].dt.dayofweek
train['weekofyear'] = train['date'].dt.isocalendar().week.astype(int)
train['is_weekend'] = train['dayofweek'].isin([5, 6]).astype(int)

test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['dayofweek'] = test['date'].dt.dayofweek
test['weekofyear'] = test['date'].dt.isocalendar().week.astype(int)
test['is_weekend'] = test['dayofweek'].isin([5, 6]).astype(int)

In [8]:
#adjust the order of dataset
train_order = [
    'id', 'date','year','month','day','dayofweek','weekofyear','is_weekend', 'holidays_locale', 'transferred', 
    'dcoilwtico',
    'store_nbr', 'city', 'state', 'type', 'cluster',
    'onpromotion', 'family', 'transactions','sales'
]

test_order = [
    'id', 'date','year','month','day','dayofweek','weekofyear','is_weekend','holidays_locale', 'transferred', 
    'dcoilwtico',
    'store_nbr',  'city', 'state', 'type', 'cluster',
   'onpromotion', 'family', 'transactions'
]

train = train[train_order]
test = test[test_order]

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import lightgbm as lgb

# split categorical and numerical columns
cat_cols = ['store_nbr', 'family', 'city', 'state', 'type', 'cluster', 'holidays_locale', 'transferred']
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    all_values = pd.concat([train[col], test[col]], axis=0).astype(str)
    le.fit(all_values)
    encoders[col] = le
    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))
    
# splite train set and validation set，dependent and independent variables
X_train = train[train['date'] < '2017-01-01'].drop(columns=['sales', 'date', 'id'])
y_train = train[train['date'] < '2017-01-01']['sales']
X_val = train[train['date'] >= '2017-01-01'].drop(columns=['sales', 'date', 'id'])
y_val = train[train['date'] >= '2017-01-01']['sales']

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

X_train.info()
X_val.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2642706 entries, 0 to 2642705
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   year             int32  
 1   month            int32  
 2   day              int32  
 3   dayofweek        int32  
 4   weekofyear       int64  
 5   is_weekend       int64  
 6   holidays_locale  int64  
 7   transferred      int64  
 8   dcoilwtico       float64
 9   store_nbr        int64  
 10  city             int64  
 11  state            int64  
 12  type             int64  
 13  cluster          int64  
 14  onpromotion      int64  
 15  family           int64  
 16  transactions     float64
dtypes: float64(2), int32(4), int64(11)
memory usage: 322.6 MB
<class 'pandas.core.frame.DataFrame'>
Index: 411642 entries, 2642706 to 3054347
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   year             411642 non-null  int32  
 1   

In [10]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 64,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

model = lgb.train(
    params,
    train_data,
    num_boost_round=2000,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=100)
    ]
)

# 验证集预测
y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)

rmse = mean_squared_error(y_val, y_val_pred, squared=False)
rmsle = np.sqrt(mean_squared_log_error(y_val, np.maximum(y_val_pred, 0)))

print(f"Validation RMSE: {rmse:.4f}")
print(f"Validation RMSLE: {rmsle:.4f}")

# 在 test 上预测
X_test = test.drop(columns=['date','id'])
y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)

# 生成提交文件
submission = pd.DataFrame({
    'id': test['id'],
    'sales': np.maximum(y_test_pred, 0)   # 销量不能为负
})

submission.to_csv('submission.csv', index=False)
print("提交文件已生成 submission.csv")

Training until validation scores don't improve for 100 rounds
[100]	train's rmse: 321.671	valid's rmse: 375.773
[200]	train's rmse: 273.641	valid's rmse: 335.942
[300]	train's rmse: 250.873	valid's rmse: 325.056
[400]	train's rmse: 236.794	valid's rmse: 319.129
[500]	train's rmse: 226.507	valid's rmse: 318.81
[600]	train's rmse: 218.462	valid's rmse: 317.416
[700]	train's rmse: 211.347	valid's rmse: 316.762
[800]	train's rmse: 205.863	valid's rmse: 317.114
Early stopping, best iteration is:
[705]	train's rmse: 211.039	valid's rmse: 316.6
Validation RMSE: 316.5996
Validation RMSLE: 1.4080
提交文件已生成 submission.csv
